In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2
import os
import pathlib
import sqlalchemy as sa
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

from tables import County, DataTrustedIdentifier, InfoImage, create_info_image
from database import connect, state_province_in_list_uf_or_list_state, has_state_in_locality
from sqlalchemy import and_, or_
from sqlalchemy.orm import Session

from unaccent import unaccent

def text_bold(string):
    return '\033[1m' + string + '\033[0m'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## database connection

In [10]:
session: Session
engine, session = connect()
engine.echo=False

2023-03-19 19:59:32,503 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-19 19:59:32,504 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 19:59:32,505 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-19 19:59:32,505 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-19 19:59:32,506 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-19 19:59:32,506 INFO sqlalchemy.engine.Engine [raw sql] {}


## BR dataset

In [11]:
county = session.query(County).distinct().all()

list_uf = [unaccent(sa.func.lower(c.uf)) for c in county]
list_state = [unaccent(sa.func.lower(c.state)) for c in county]
list_county = [unaccent(sa.func.lower(c.county)) for c in county]

query_lower_unaccent_like = session.query(sa.func.concat('%', unaccent(sa.func.lower(County.state)), '%')).distinct().all()
list_state_like = [q[0] for q in query_lower_unaccent_like]
columns = [DataTrustedIdentifier.specific_epithet, DataTrustedIdentifier.barcode]

/tmp/ipykernel_544885/1285570762.py:7: SAWarning: Class unaccent will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/20/cprf)
  query_lower_unaccent_like = session.query(sa.func.concat('%', unaccent(sa.func.lower(County.state)), '%')).distinct().all()


In [12]:
query_only_state = session.query(*columns)\
    .filter(and_(DataTrustedIdentifier.country_trusted =='Brasil',
                DataTrustedIdentifier.specific_epithet.is_not(None),
                state_province_in_list_uf_or_list_state(list_state, list_uf)))\
    .distinct()\
    .all()

print('count of records using only field %s: %d' % (text_bold('state_province'), len(query_only_state)))

quantidade de registros usando somente state_province: 11914


In [66]:
query_has_state_in_locality = session.query(*columns)\
    .filter(and_(DataTrustedIdentifier.country_trusted=='Brasil',
                 DataTrustedIdentifier.specific_epithet.is_not(None),
                 or_(state_province_in_list_uf_or_list_state(list_state, list_uf),
                     has_state_in_locality(list_state_like))))\
    .distinct()\
    .all()

print('count of records using only fields %s: %d' % (text_bold('state_province and locality'), len(query_has_state_in_locality)))

count of records using only fields state_province and locality: 11914


In [24]:
columns = [DataTrustedIdentifier.specific_epithet, sa.func.array_agg(DataTrustedIdentifier.seq)]

qz = session.query(*columns)\
    .filter(and_(DataTrustedIdentifier.country_trusted =='Brasil',
                DataTrustedIdentifier.specific_epithet.is_not(None),
                state_province_in_list_uf_or_list_state(list_state, list_uf)))\
    .distinct()\
    .group_by(DataTrustedIdentifier.specific_epithet)\
    .order_by(DataTrustedIdentifier.specific_epithet)\
    .having(sa.func.count(DataTrustedIdentifier.specific_epithet) >= 5)\
    .all()

print('count of species: %d' % len(qz))

247

In [25]:
import pandas as pd

list_images_invalid = {
    'barcode': ['INPA0248526', 'INPA0248523', 'INPA0248528', 'NY01421575_01', 'HUFSJ001689_v00', 'HUFSJ001133_v00', 'HUFSJ002198_v00', 'HUFSJ003255_v00', 'HVASF000487_v01', 'INPA0019084_nd', 'INPA0022379_nd', 'INPA0032742_nd', 'INPA0023115', 'NL-U1484137', 'INPA0012286', 'INPA0146998'],
    'reason': ['horizontal', 'horizontal', 'horizontal', 'horizontal', 'not exsicate', 'not exsicate', 'not exsicate', 'not exsicate', 'not exsicate', 'label', 'label', 'label', 'letter', 'letter', 'letter', 'incomplete']
}

pd.DataFrame(list_images_invalid)

,barcode,reason
0,INPA0248526,horizontal
1,INPA0248523,horizontal
2,INPA0248528,horizontal
3,NY01421575_01,horizontal
4,HUFSJ001689_v00,not exsicate
5,HUFSJ001133_v00,not exsicate
6,HUFSJ002198_v00,not exsicate
7,HUFSJ003255_v00,not exsicate
8,HVASF000487_v01,not exsicate
9,INPA0019084_nd,label


In [48]:
import itertools
import numpy as np
import pandas as pd

l_species = []
l_paths = []
lc = []
color = 'rgb'
image_size='512'
sum = 0
sumb = 0
for i, qq in enumerate(qz):
    species = qq[0]
    list_seq = qq[1]
    qzinho = session.query(sa.func.array_agg(sa.distinct(InfoImage.path_image))) \
        .filter(and_(InfoImage.seq_id.in_(list_seq),
                     InfoImage.image_size == image_size,
                     InfoImage.color_mode == color)) \
        .group_by(InfoImage.seq_id) \
        .all()

    lsa = list(itertools.chain(*qzinho)) # "remove of tuples"

    if len(lsa) >= 5:
    # remove images duplicates
        lb=[]
        for lx in lsa:
            lb.append(sorted(lx)[0]) # sorted and catch first value of list

        if len(np.unique(lb)) >= 5:
            l_species.append(species)
            l_paths.append(np.unique(lb).tolist())
            lc.append(len(np.unique(lb)))
            print(lb)
            sumb += len(np.unique(lb))
            sum += 1
    # break

sum, sumb

['/home/xandao/Documentos/RGB/512/w_pred_mask/MBM084431.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/MBM033212.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/MBM033181.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/CEN00051607.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/SP011164.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY00555584_01.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/NY00555583_01.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/INPA0124667.jpeg']
['/home/xandao/Documentos/RGB/512/w_pred_mask/UEC149671.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/UEC149670.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/UEC149669.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/UEC138168.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/UEC149653.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/UEC149673.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/UEC149668.jpeg', '/home/xandao/Documentos/RGB/512/w_pred_mask/MBM116015.jp

(235, 10528)

In [49]:
pdx = {'s': l_species, 'c': lc}
pd.DataFrame(pdx)

,s,c
0,abutiloides,8
1,aduncum,377
2,aequale,30
3,alata,81
4,alatabaccum,28
...,...,...
230,viminifolium,29
231,vitaceum,13
232,wachenheimii,5
233,warmingii,6


In [50]:
sum = 0
for pqwq in l_paths:
    sum += len(pqwq)
len(l_species), sum

(235, 10528)

In [51]:
sum = 0
for pqwq in l_paths:
    if len(pqwq) >= 5:
        sum += 1
sum

235

In [61]:
l_final = []
l_count = []
list_removed = []
sum = 0
for i, s in enumerate(l_paths):
    matching = [sx for sx in s if not any(xs in sx for xs in list_images_invalid['barcode'])]

    if len(s) != len(matching):
        print('specie: %s before: %d after: %d' % (l_species[i], len(s), len(matching)))
        diff = list(set(s) ^ set(matching))
        print('diff: %s' % str(diff))

    sum += len(matching)
    l_final.append(matching)
    l_count.append(len(matching))

print('total of species species: %d ')

specie: aleyreanum before: 54 after: 53
['/home/xandao/Documentos/RGB/512/w_pred_mask/INPA0023115.jpeg']
specie: arboreum before: 621 after: 620
['/home/xandao/Documentos/RGB/512/w_pred_mask/INPA0248528.jpeg']
specie: bartlingianum before: 146 after: 145
['/home/xandao/Documentos/RGB/512/w_pred_mask/NY01421575_01.jpeg']
specie: belterraense before: 14 after: 13
['/home/xandao/Documentos/RGB/512/w_pred_mask/NL-U1484137.jpeg']
specie: blanda before: 114 after: 113
['/home/xandao/Documentos/RGB/512/w_pred_mask/HUFSJ001133_v00.jpeg']
specie: callosum before: 60 after: 59
['/home/xandao/Documentos/RGB/512/w_pred_mask/INPA0022379_nd.jpeg']
specie: decora before: 39 after: 38
['/home/xandao/Documentos/RGB/512/w_pred_mask/HUFSJ002198_v00.jpeg']
specie: hispidum before: 332 after: 331
['/home/xandao/Documentos/RGB/512/w_pred_mask/INPA0032742_nd.jpeg']
specie: hostmannianum before: 146 after: 144
['/home/xandao/Documentos/RGB/512/w_pred_mask/INPA0248526.jpeg', '/home/xandao/Documentos/RGB/512/w_

(235, 10512)

In [64]:
pd.DataFrame({
    'species': l_species,
    'paths': l_final,
    'count': l_count
})

,species,paths,count
0,abutiloides,[/home/xandao/Documentos/RGB/512/w_pred_mask/C...,8
1,aduncum,[/home/xandao/Documentos/RGB/512/w_pred_mask/A...,377
2,aequale,[/home/xandao/Documentos/RGB/512/w_pred_mask/C...,30
3,alata,[/home/xandao/Documentos/RGB/512/w_pred_mask/C...,81
4,alatabaccum,[/home/xandao/Documentos/RGB/512/w_pred_mask/I...,28
...,...,...,...
230,viminifolium,[/home/xandao/Documentos/RGB/512/w_pred_mask/C...,29
231,vitaceum,[/home/xandao/Documentos/RGB/512/w_pred_mask/I...,13
232,wachenheimii,[/home/xandao/Documentos/RGB/512/w_pred_mask/N...,5
233,warmingii,[/home/xandao/Documentos/RGB/512/w_pred_mask/M...,6


In [65]:
print('total of images: %d' % np.sum(l_count))

total of images: 10512
